<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
这是<a href="http://mng.bz/orYv">从零开始构建一个大语言模型</a>这本书的补充代码。 作者 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>代码仓库: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 通过Llama 3和Ollama生成指令数据集

- 本笔记本使用80亿参数的Llama 3模型通过ollama生成一个合成数据集，使用的是"Magpie: Alignment Data Synthesis from Scratch by Prompting Aligned LLMs with Nothing"论文中提出的"hack"方法([https://arxiv.org/abs/2406.08464](https://arxiv.org/abs/2406.08464))
- 生成的数据集将是一个指令数据集，包含"instruction"和"output"字段，类似于Alpaca中的内容：


```python
{
    "instruction": "What is the atomic number of helium?",
    "output": "The atomic number of helium is 2.",
},
```

- 这段代码不需要GPU,可以在笔记本电脑上运行（已在M3 MacBook Air上测试过）

*请注意,这里创建的指令数据集仅用于教育目的。然而,用户有责任确保他们的使用符合与Meta AI的Llama 3相关的许可协议条款。*

In [1]:
from importlib.metadata import version

pkgs = [
    "tqdm",    # Progress bar
]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tqdm version: 4.66.4


## 安装ollama并下载Llama 3

- Ollama是一个应用程序，用于高效地运行LLMs
- 它是一个围绕[llama.cpp](https://github.com/ggerganov/llama.cpp)的包装器，后者在纯C/C++中实现了LLMs，以最大化效率
- 请注意，它是一个用于使用LLMs生成文本（推理）的工具，而不是用于训练或微调LLMs的工具
- 在运行下面的代码之前，请访问[https://ollama.com](https://ollama.com)并按照说明进行安装（例如，点击“下载”按钮并下载适用于您的操作系统的ollama应用程序）


- 对于macOS和Windows用户，点击您下载的ollama应用程序；如果它提示您安装命令行使用，请说“是”
- Linux用户可以在ollama网站上使用提供的安装命令

- 通常，在我们能够从命令行使用ollama之前，我们必须启动ollama应用程序或在一个单独的终端中运行`ollama serve`

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/ollama-eval/ollama-serve.webp?1">


- 在ollama应用程序或`ollama serve`运行时，在不同的终端中，在命令行中执行以下命令，尝试使用80亿参数的Llama 3模型（该模型占用4.7 GB的存储空间，第一次执行此命令时会自动下载）

```bash
# 8B model
ollama run llama3
```


输出如下所示：

```
$ ollama run llama3
pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 
```

- 请注意，`llama3`指的是指令微调的80亿参数的Llama 3模型

- 您还可以使用更大的70亿参数的Llama 3模型，如果您的机器支持它，请将`llama3`替换为`llama3:70b`

- 下载完成后，您将看到一个命令行提示符，允许您与模型进行交互

- 尝试一个提示，如"What do llamas eat?", 这将返回一个类似于以下内容的输出：

```
>>> What do llamas eat?
Llamas are ruminant animals, which means they have a four-chambered 
stomach and eat plants that are high in fiber. In the wild, llamas 
typically feed on:
1. Grasses: They love to graze on various types of grasses, including tall 
grasses, wheat, oats, and barley.
```

- 您可以使用输入`/bye`结束此会话

## 使用Ollama的REST API

- 现在，另一种与模型交互的方式是通过Python中的Ollama的REST API，如下面的函数所示
- 在运行此笔记本中的下一个单元格之前，请确保ollama仍在运行，如上所述，通过
  - 在终端中运行`ollama serve` 启动ollama程序
- 接下来，运行以下单元格代码以查询模型

- 首先，让我们用一个简单的例子来测试API，以确保它按预期工作：

In [2]:
import urllib.request
import json

def query_model(prompt, model="llama3", url="http://localhost:11434/api/chat", role="user"):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "seed": 123,        # for deterministic responses
        "temperature": 1.,   # for deterministic responses
        "top_p": 1,         
        "messages": [
            {"role": role, "content": prompt}
        ]
    }

    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data

In [3]:
result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet typically consists of:

1. Grasses: Llamas love to graze on various types of grasses, including tall grasses, short grasses, and even weeds.
2. Hay: They enjoy eating hay, such as alfalfa or timothy hay, which provides them with fiber, protein, and other essential nutrients.
3. Grains: Llamas may eat grains like oats, barley, or corn as a supplement to their diet.
4. Leaves: They will also munch on leaves from trees and shrubs, including clover, alfalfa, and various types of leaves.
5. Fruits and vegetables: In the wild, llamas might eat fruits and vegetables that grow in their natural habitat, such as apples, carrots, or potatoes.

In general, a llama's diet should consist of:

* 50% grasses and hay
* 20% grains (like oats or corn)
* 10% leaves and other plant material
* 5% fruits and vegetables (as treats)

It's essential to provide llamas with a balanced diet that meets their nutritional n

## 提取指令

- 现在,让我们使用论文中提出的"技巧"：我们提供空的提示模板`"<|begin_of_text|><|start_header_id|>user<|end_header_id|>"`，这将导致经过指令微调的Llama 3模型生成一条指令

In [4]:
def extract_instruction(text):
    for content in text.split("\n"):
        if content:
            return content.strip()

In [5]:
query = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>"

result = query_model(query, role="assistant")
instruction = extract_instruction(result)
print(instruction)

I am trying to find a way to make my child's birthday party more special and unique. What are some creative ideas you have?


- 我们可以看到，令人惊讶的是，模型确实生成了一个指令

## 生成响应

- 现在，下一步是创建相应的响应，这可以通过简单地将指令作为输入来完成

In [6]:
response = query_model(instruction, role="user")
print(response)

What an exciting question! I'd be delighted to help you come up with some creative and unique ideas to make your child's birthday party truly special!

Here are a few ideas to get you started:

1. **Themed Scavenger Hunt**: Plan a scavenger hunt based on the birthday child's favorite theme (e.g., superheroes, animals, or princesses). Hide clues and challenges throughout the party area, leading up to a final surprise.
2. **DIY Crafts Station**: Set up a craft station where kids can create their own party favors, such as customized t-shirts, crowns, or jewelry. This activity encourages creativity and makes for a memorable keepsake.
3. **Mystery Box Challenge**: Fill mystery boxes with different textures, smells, and sounds. Have the kids guess what's inside each box without looking. This game promotes problem-solving and teamwork.
4. **Indoor Camping Adventure**: Set up a cozy indoor "camping" area with sleeping bags, flashlights, and s'mores-making stations. Kids can enjoy a camping exp

## 生成数据集

- 我们可以将这种方法扩展到任意数量的数据样本（您可能想要应用一些可选的过滤长度或质量（例如，使用另一个LLM来评估生成的数据）
- 下面，我们生成5个合成指令-响应对，这大约需要3分钟在M3 MacBook Air上
- (要生成适合指令微调的数据集，我们希望将其增加到至少1k到50k，并可能在其上运行GPU以更及时地生成示例)

**Tip**

- 您可以通过将`model="llama3"`更改为`model="llama3:70b"`来生成更高质量的响应，然而，这需要更多的计算资源

In [7]:
from tqdm import tqdm

dataset_size = 5
dataset = []

for i in tqdm(range(dataset_size)):

    result = query_model(query, role="assistant")
    instruction = extract_instruction(result)
    response = query_model(instruction, role="user")
    entry = {
        "instruction": instruction,
        "output": response
    }
    dataset.append(entry)

100%|█████████████████████████████████████████████| 5/5 [02:37<00:00, 31.41s/it]


In [8]:
with open("instruction-data-llama3-7b.json", "w") as file:
    json.dump(dataset, file, indent=4)

In [9]:
!cat instruction-data-llama3-7b.json

[
    {
        "instruction": "What is the significance of the number 7 in various cultures and religions?",
        "output": "The number 7 has been a significant and recurring theme across many cultures and religions, often imbuing it with special meaning and symbolism. Here are some examples:\n\n1. **Numerology**: In numerology, the number 7 is considered sacred and mystical, associated with spiritual awakening, introspection, and enlightenment.\n2. **Judaism**: The Torah has seven days of creation, seven weeks in the wilderness, and seven years of rest (Sabbatical year). Seven is also a symbol of completion or perfection.\n3. **Christianity**: In Christianity, there are seven deadly sins, seven virtues, and seven sacraments. Jesus was said to have spoken seven sermons, and the number 7 appears in various biblical accounts, such as the seven days of creation and the seven angels who appear before God.\n4. **Islam**: In Islamic tradition, there are seven heavens, seven earths, and s